In [6]:
import tasks
from utils import parse_config

configuration = parse_config("config")

raw_data = configuration["raw_data"]
TABLES = configuration["TABLES"]
FACT_TABLES = configuration["FACT_TABLES"]
SCD_types = configuration["SCD"]

db_master = configuration["db_master"]

db_rel = configuration["db_rel"]
db_dim = configuration["db_dim"]

schema_rel = configuration["schema_rel"]
schema_dim = configuration["schema_dim"]

In [ ]:
conn_master = tasks.connect_to(db_master)

tasks.drop_database_if_exists(conn_master)
tasks.create_database(conn_master)

conn_master.close()

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=master;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    



In [ ]:
conn_ER = tasks.connect_to(db_rel)

tasks.create_database(conn_ER)

tasks.drop_constraints_if_exist(conn_ER, db_rel, schema_rel)

for table in TABLES:
    tasks.drop_table_if_exists(conn_ER, table, db_rel, schema_rel)
    tasks.create_table(conn_ER, table, db_rel, schema_rel)
    tasks.insert_into_table(conn_ER, raw_data, table, db_rel, schema_rel)

tasks.set_constraints(conn_ER, db_rel, schema_rel)

conn_ER.close()

In [ ]:
conn_DW = tasks.connect_to(db_dim)

for table, SCD_type in SCD_types.items():
    
    tasks.drop_dim_table_if_exists(conn_DW, table, SCD_type, db_dim, schema_dim)
    
    tasks.create_dim_table(conn_DW, table, SCD_type, db_dim, schema_dim)
    
    tasks.update_dim_table(conn_DW, table, SCD_type, db_dim, db_rel, schema_dim, schema_rel)


conn_DW.close()

In [ ]:
conn_DW = tasks.connect_to(db_dim)
for fact_table in FACT_TABLES:
    tasks.drop_fact_table_if_exists(conn_DW, fact_table, db_dim, schema_dim)
    tasks.create_fact_table(conn_DW, fact_table, db_dim, schema_dim)
    tasks.update_fact_table(conn_DW, fact_table, db_dim, db_rel, schema_dim, schema_rel)
    
conn_DW.close()